In [8]:
import os

%pwd


'/Users/satwiksahoo/Desktop/CodeBasics/machine learning/krish naik/NLP project/ATSresume/RESUME_ATS'

In [ ]:
# os.chdir('../')

In [9]:
# from dataclasses import dataclass
# from pathlib import Path

# @dataclass
# class ModelTrainerConfig:
#     root_dir : Path
#     transformed_train_path : Path
#     transformed_test_path : Path
#     trained_model_artifact_path : Path
#     output_dir: Path
#     eval_strategy : str         # evaluate each epoch
#     save_strategy : str      # save best model each epoch
#     learning_rate: float        # slightly higher than 2e-5 (good for smaller datasets)
#     per_device_train_batch_size: int # larger batch = stabler gradient updates
#     per_device_eval_batch_size: int
#     num_train_epochs: int            # train a bit longer, but use early stopping
#     weight_decay: float              # keep small L2 regularization
#     # logging_dir= "./logs"
#     logging_steps: int
#     save_total_limit: int             # keep only last 2 checkpoints
#     # load_best_model_at_end:True    # reload best model based on metric
#     metric_for_best_model: str     # pick best model using F1 (good for imbalance)
#     # greater_is_better:True
#     warmup_ratio: float               # gradual warmup → smoother start
#     lr_scheduler_type: str     # cosine schedule → better convergence
#     gradient_accumulation_steps: int  # effective batch size = 16*2 = 32
#     # fp16: True                      # if GPU supports mixed precision (saves memory)
#     seed: int                         # reproducibility


# config_entity.py
from dataclasses import dataclass

@dataclass
class ModelTrainerConfig:
    root_dir: str
    transformed_train_path: str
    transformed_test_path: str
    trained_model_artifact_path: str
    output_dir: str
    eval_strategy: str
    save_strategy: str
    learning_rate: float
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    num_train_epochs: int
    weight_decay: float
    logging_dir: str
    logging_steps: int
    save_total_limit: int
    load_best_model_at_end: bool
    metric_for_best_model: str
    greater_is_better: bool
    warmup_ratio: float
    lr_scheduler_type: str
    gradient_accumulation_steps: int
    fp16: bool
    seed: int


In [10]:
from src.ATS_RESUME.constants import *
from src.ATS_RESUME.utils.common import read_yaml , create_directories

In [11]:

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            transformed_train_path=config.transformed_train_path,
            transformed_test_path=config.transformed_test_path,
            trained_model_artifact_path=config.trained_model_artifact_path,  # FIXED
            output_dir=params.output_dir,
            eval_strategy=params.eval_strategy,
            save_strategy=params.save_strategy,
            learning_rate=params.learning_rate,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            num_train_epochs=params.num_train_epochs,
            weight_decay=params.weight_decay,
            logging_dir="./logs",
            logging_steps=params.logging_steps,
            save_total_limit=params.save_total_limit,
            load_best_model_at_end=params.load_best_model_at_end,
            metric_for_best_model=params.metric_for_best_model,
            greater_is_better=params.greater_is_better,
            warmup_ratio=params.warmup_ratio,
            lr_scheduler_type=params.lr_scheduler_type,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            fp16=params.fp16,
            seed=params.seed
        )
        return model_trainer_config


In [93]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer, InputExample, losses, models, evaluation
from torch.utils.data import DataLoader
import pandas as pd
from datasets import load_dataset
import pickle
import os

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def initiate_model_trainer_(self):
        # Load tokenized train/test data
        train_dataset = load_from_disk(self.config.transformed_train_path)
        test_dataset = load_from_disk(self.config.transformed_test_path)

        # Load tokenizer + model
        model_name = "distilbert-base-uncased"   # <--- change as needed
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

        # Training args
        training_args = TrainingArguments(
            output_dir=self.config.output_dir,
            eval_strategy=self.config.eval_strategy,
            save_strategy=self.config.save_strategy,
            learning_rate=self.config.learning_rate,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            num_train_epochs=self.config.num_train_epochs,
            weight_decay=self.config.weight_decay,
            logging_dir=self.config.logging_dir,
            logging_steps=self.config.logging_steps,
            save_total_limit=self.config.save_total_limit,
            load_best_model_at_end=self.config.load_best_model_at_end,
            metric_for_best_model=self.config.metric_for_best_model,
            greater_is_better=self.config.greater_is_better,
            warmup_ratio=self.config.warmup_ratio,
            lr_scheduler_type=self.config.lr_scheduler_type,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            fp16=self.config.fp16,
            seed=self.config.seed
        )

        # Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            tokenizer=tokenizer
        )

        trainer.train()
        model.save_pretrained(self.config.trained_model_artifact_path)
        tokenizer.save_pretrained(self.config.trained_model_artifact_path)
    
    def initiate_model_trainer(self):
        train_file = os.path.join(self.config.transformed_train_path, "train.pkl")
        test_file = os.path.join(self.config.transformed_test_path, "test.pkl")
        
        with open(train_file, "rb") as f:
          train_data = pickle.load(f)

        with open(test_file, "rb") as f:
           val_data = pickle.load(f)

        
        
        train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)
        test_dataloader = DataLoader(val_data, shuffle=False, batch_size=16)

# Load pre-trained bi-encoder
        model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

# Loss: Cosine similarity loss (regression on similarity scores)
        train_loss = losses.CosineSimilarityLoss(model)

# Evaluator: compute correlation between predicted similarity & true labels
        evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_data, name="val")

# Train
        model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=6,
    warmup_steps=100,
    evaluation_steps=500,
    output_path="./bi_encoder_ats_model"
)

        


In [12]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config = model_trainer_config)
model_trainer.initiate_model_trainer()


[2025-08-30 11:38:34,722: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-30 11:38:34,727: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-30 11:38:34,727: INFO: common: created directory at: artifacts]
[2025-08-30 11:38:34,729: INFO: common: created directory at: artifacts/model_trainer]


NameError: name 'ModelTrainer' is not defined

In [74]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer , pipeline
# from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained("artifacts/model_trainer/trained_model_artifact")
tokenizer = AutoTokenizer.from_pretrained("artifacts/model_trainer/trained_model_artifact")



In [ ]:
# import re
# def preprocess_text(text):
#   tokens = []
#   text = re.sub(r'\r\n|\n', ' ', text)
#   text = re.sub(r'\s+', ' ', text)
#   text = re.sub(r'<.*?>', ' ', text)
#   text = text.strip().lower()



#   return text



In [ ]:
# classifier = pipeline('text-classification' , model = model , tokenizer=tokenizer)


# resume_text = """
# Data Scientist with 6 years of experience in Python, SQL, and machine learning.
# Worked on predictive modeling, NLP, and recommendation systems. Skilled in TensorFlow,
# PyTorch, and cloud deployment.
# """

# job_text = """
# Looking for a Senior Data Scientist with 5+ years of experience in predictive analytics,
# machine learning, and deep learning. Strong Python and SQL skills required. 
# Experience with cloud ML deployment is a plus.
# """

# # Concatenate resume and job description as input
# test_input = preprocess_text(resume_text) + " [SEP] " + preprocess_text(job_text) # /// q

# prediction = classifier(test_input)
# print(prediction)

# mapping = {  'LABEL_0':'No Fit'  ,  'LABEL_1' : 'Potential Fit' ,  'LABEL_2' : 'Good Fit'   }

# print(mapping[prediction[0]['label']])


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.9997040629386902}]
No Fit


In [ ]:
# print(model.config.id2label)
# 

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}


In [81]:
# classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# resume_text = """
# Data Scientist with 6 years of experience in Python, SQL, and machine learning.
# Worked on predictive modeling, NLP, and recommendation systems. Skilled in TensorFlow,
# PyTorch, and cloud deployment.
# """

# job_text = """
# Looking for a Senior Data Scientist with 5+ years of experience in predictive analytics,
# machine learning, and deep learning. Strong Python and SQL skills required. 
# Experience with cloud ML deployment is a plus.
# """

# prediction = classifier((resume_text, job_text))   # let tokenizer handle [SEP]
# print(prediction)

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # # Load tokenizer and model
# # tokenizer = AutoTokenizer.from_pretrained("path/to/your/model")
# # model = AutoModelForSequenceClassification.from_pretrained("path/to/your/model")
# model = AutoModelForSequenceClassification.from_pretrained("artifacts/model_trainer/trained_model_artifact")
# tokenizer = AutoTokenizer.from_pretrained("artifacts/model_trainer/trained_model_artifact")
# resume_text = """
# Data Scientist with 6 years of experience in Python, SQL, and machine learning.
# Worked on predictive modeling, NLP, and recommendation systems. Skilled in TensorFlow,
# PyTorch, and cloud deployment.
# """

# job_text = """
# Looking for a Senior Data Scientist with 5+ years of experience in predictive analytics,
# machine learning, and deep learning. Strong Python and SQL skills required. 
# Experience with cloud ML deployment is a plus.
# """

# # Concatenate resume and job description
# test_input = preprocess_text(resume_text) + " [SEP] " + preprocess_text(job_text)

# # Tokenize
# inputs = tokenizer(test_input, return_tensors="pt", truncation=True, padding=True)

# # Forward pass
# with torch.no_grad():
#     outputs = model(**inputs)

# # Get logits
# logits = outputs.logits

# # Apply softmax to get probabilities
# probs = torch.nn.functional.softmax(logits, dim=-1)

# # Get predicted label index
# predicted_class_id = torch.argmax(probs, dim=-1).item()

# # Map id -> label
# id2label = model.config.id2label  # e.g., {0: "LABEL_0", 1: "LABEL_1", 2: "LABEL_2"}

# # Custom mapping
# mapping = {
#     "LABEL_0": "No Fit",
#     "LABEL_1": "Potential Fit",
#     "LABEL_2": "Good Fit"
# }

# print("Prediction:", mapping[id2label[predicted_class_id]])
# print("Probabilities:", probs.tolist())


Prediction: No Fit
Probabilities: [[0.9997040629386902, 0.00024037726689130068, 5.5546257499372587e-05]]


In [14]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, evaluation
from torch.utils.data import DataLoader
import pandas as pd
from datasets import load_dataset


from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("artifacts/model_trainer/model5")


resume = """
Marketing Specialist with 7 years of experience in SEO, content marketing,
and social media strategy. Recently completed a data analytics certification 
with hands-on projects in Python and SQL.
"""

jd = """
Looking for a Senior Data Scientist with 5+ years of experience in predictive analytics,
machine learning, and deep learning. Strong Python and SQL skills required. 
Experience with cloud ML deployment is a plus.
"""
emb_resume = model.encode(resume, convert_to_tensor=True)
emb_jd = model.encode(jd, convert_to_tensor=True)

similarity = util.cos_sim(emb_resume, emb_jd).item()
print(f"Similarity Score: {similarity:.4f}")

# Thresholds (tune with validation set)
if similarity < 0.4:
    label = "No Fit"
elif similarity < 0.7:
    label = "Potential Fit"
else:
    label = "Good Fit"

print(f"Predicted Label: {label}")


[2025-08-30 11:42:40,481: INFO: SentenceTransformer: Use pytorch device_name: mps]
[2025-08-30 11:42:40,482: INFO: SentenceTransformer: Load pretrained SentenceTransformer: artifacts/model_trainer/model5]


Batches: 100%|██████████| 1/1 [00:00<00:00, 58.86it/s]

Similarity Score: 0.7148
Predicted Label: Good Fit
